# Defines the complete model in David's paper

## Simulate single trajectory up to X level

In [1]:
import pymc3 as pm
import numpy as np
M = 6
K = 10
D = 250
Tn = 20
step_sizes = [1,2,4]

import rpy2
%load_ext rpy2.ipython

In [2]:
%Rpush Tn M

In [3]:
%%R
library(smfsb)
#pi <- c(0.147026,0.102571,0.239819,0.188710,0.267137,0.054738)
pi <- c(1,0,0,0,0,0)

Q <- matrix(c(-0.631921,0.631921,0.000000,0.000000,0.000000,0.000000,
    0.000000,-0.229485,0.229485,0.000000,0.000000,0.000000,
    0.000000,0.000000,-0.450538,0.450538,0.000000,0.000000,
    0.000000,0.000000,0.000000,-0.206042,0.206042,0.000000,
    0.000000,0.000000,0.000000,0.000000,-0.609582,0.609582,
    0.000000,0.000000,0.000000,0.000000,0.00001,-0.00001), nrow=M, ncol=M,byrow=TRUE)

set.seed(1729)
s<-rcfmc(1e3, Q, pi)

In [4]:
step_sizes = [1,2,4]
np.random.seed(1729)
observed_jumps = np.random.choice(step_sizes, Tn-1)
observed_times = np.insert(np.cumsum(observed_jumps),0,0)
%Rpush observed_times

In [5]:
%R S <- s(observed_times)-1

array([ 0.,  1.,  1.,  2.,  2.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  5.,  5.,  5.])

In [6]:
%Rpull S
print len(S)
print len(observed_jumps)
print observed_jumps
print observed_times
print S

20
19
[2 4 2 2 4 2 2 2 1 2 2 2 4 1 1 1 1 4 2]
[ 0  2  6  8 10 14 16 18 20 21 23 25 27 31 32 33 34 35 39 41]
[ 0.  1.  1.  2.  2.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.]


In [7]:
len(observed_jumps)
#len(S)

19

In [8]:
B = np.array([[0.000001,0.760000,0.720000,0.570000,0.700000,0.610000],
[0.000001,0.460000,0.390000,0.220000,0.200000,0.140000],
[0.000001,0.620000,0.620000,0.440000,0.390000,0.240000],
[0.000001,0.270000,0.210000,0.170000,0.190000,0.070000],
[0.000001,0.490000,0.340000,0.220000,0.160000,0.090000],
[0.000001,0.620000,0.340000,0.320000,0.240000,0.120000],
[0.000001,0.550000,0.390000,0.320000,0.290000,0.150000],
[0.000001,0.420000,0.240000,0.170000,0.170000,0.110000],
[0.000001,0.310000,0.300000,0.230000,0.190000,0.110000],
[0.000001,0.470000,0.340000,0.190000,0.190000,0.110000]])

B0 = np.array([[0.410412,0.410412,0.418293,0.418293,0.429890,0.429890],
[0.240983,0.240983,0.240983,0.240983,0.240983,0.240983],
[0.339714,0.339714,0.339714,0.339714,0.339714,0.339714],
[0.130415,0.130415,0.130415,0.130415,0.130415,0.130415],
[0.143260,0.143260,0.143260,0.143260,0.143260,0.143260],
[0.211465,0.211465,0.211465,0.211465,0.211465,0.211465],
[0.194187,0.194187,0.194187,0.194187,0.194187,0.194187],
[0.185422,0.185422,0.185422,0.185422,0.185422,0.185422],
[0.171973,0.171973,0.171973,0.171973,0.171973,0.171973],
[0.152277,0.152277,0.152277,0.152277,0.152277,0.152277]])

X_input = np.zeros((K, Tn))
for k in range(K):
    X_input[k,0] = np.random.binomial(n=1, p=B0[k,S[0]] )
for i in range(1, len(S)-1):
    for k in range(K):
        if S[i] == S[i-1] or X_input[k,i-1] > 0.5:
            X_input[k,i] = X_input[k,i-1]
        else:
            X_input[k,i] = np.random.binomial(n=1, p=B[k,S[i]])

X_input

array([[ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  

## Auxilary Distributions

In [9]:
from pymc3 import Continuous
from pymc3.distributions import transforms
import theano.tensor as T
from theano.tensor.nlinalg import eig, matrix_inverse
from theano.compile.sharedvalue import shared

#prior
class DiscreteObsMJP_unif_prior(Continuous):

    def __init__(self, n, transform=transforms.log, *args, **kwargs):
        super(DiscreteObsMJP_unif_prior, self).__init__(transform=transform, *args, **kwargs)
        Q_raw = np.ones((n,n-1))
        self.n = n
        self.mode = Q_raw

    def logp(self, value):
        return 0

#likelihood
class DiscreteObsMJP(Continuous):

    def __init__(self, Q_raw, S, observed_jumps, *args, **kwargs):
        super(DiscreteObsMJP, self).__init__(*args, **kwargs)
        self.Q_raw = Q_raw
        self.S = S
        self.observed_jumps = observed_jumps
        self.step_sizes = np.unique(observed_jumps)
        
        self.mode=0

    #@as_op(itypes=[T.dmatrix], otypes=[T.cmatrix])
    def convertFromRaw(self, Q_raw):

        #get integer value and symbolic representation
        nrows, ncols = Q_raw.transformed.dshape
        nrows_s, ncols_s = Q_raw.shape 

        #create Q
        Q = T.alloc(0.0, *(nrows_s, ncols_s+1))
        Q.name = 'Q'

        #fill Q
        row_sums = Q_raw.sum(axis=1)
        for i in range(0, nrows):
            ind = 0
            for j in range(0, ncols+1):
                if i==j:
                    Q = T.set_subtensor(Q[i,j], -row_sums[i])
                else:
                    Q = T.set_subtensor(Q[i,j], Q_raw[i,ind])
                    ind += 1
                    
        return Q
    
    def computeC(self):
        S = self.S
        step_sizes = self.step_sizes
        observed_jumps = self.observed_jumps
        n_step_sizes = len(self.step_sizes)
        
        nrows_s, ncols_s = Q_raw.shape
        n_step_sizes_s=shared(value=n_step_sizes, name='n_step_sizes_s')
        
        #create C
        C_true = T.alloc(0, *(nrows_s, ncols_s+1, n_step_sizes_s))
        C_true.name = 'C_true'
        
        #fill C
        for i in range(len(observed_jumps)):
            tau = observed_jumps[i]
            tau_index = 0
            for j in range(len(step_sizes)):
                if tau == step_sizes[j]:
                    tau_index = j
                    break
            #tau_index = np.where(step_sizes==tau)[0].item()
            
            Ci = S[i]
            Cj = S[i+1]
            C_true = T.set_subtensor(C_true[Ci, Cj, tau_index], 
                                     C_true[Ci, Cj, tau_index]+1)
        
        return C_true
    
    def logp(self, C):
        Q_raw = self.Q_raw
        step_sizes = self.step_sizes

        Q = self.convertFromRaw(Q_raw)
        Q_complex = T.cast(Q, 'complex64')

        C_true = self.computeC()
        
        l = 0.0
        for i in range(0, len(step_sizes)):
            #get P(tau)
            lambdas, U = eig(Q_complex)
            
            tau = step_sizes[i]
            exp_tD = T.diag(T.exp(tau*lambdas))

            U_inv = matrix_inverse(U)

            P = U.dot(exp_tD).dot(U_inv)
        
            #compute likelihood in terms of P(tau)
            l += T.sum(C_true[:,:,i]*T.log(P))
            
        return l

In [10]:
from pymc3 import Metropolis, Continuous, Beta

class Comorbidities(Continuous):
    def __init__(self, S, B, shape, *args, **kwargs):
        super(Comorbidities, self).__init__(*args, **kwargs)
        X = np.ones(shape)
        self.shape = shape
        self.S = S
        self.B = B
        self.mode = X

    def logp(self, value):
        return 0



class HMM_Blank(Continuous):
    def __init__(self, shape, *args, **kwargs):
        super(HMM_Blank, self).__init__(shape = shape, dtype='int32', *args, **kwargs)
        S = np.ones(shape, dtype=np.int32)
        self.shape = shape
        self.mode = S

    def logp(self, value):
        return 0


class Observations(Continuous):
    pass

class sampleS(Metropolis):
    pass

class sampleS(Metropolis):
    pass

## Auxilary Samplers

In [11]:
'''
Created on May 12, 2012

@author: john
'''
from pymc3.core import *
from pymc3.step_methods.arraystep import ArrayStepShared
from numpy import array, max, exp, cumsum, nested_iters, empty, searchsorted, ones
from numpy.random import uniform

from theano.gof.graph import inputs
from theano.tensor import add 
from pymc3.theanof import make_shared_replacements


class sampleS(ArrayStepShared):
    """
    Use forward sampling (equation 10) to sample a realization of S_t, t=1,...,T_n
    given Q, B, and X constant.
    """
    # TODO: It would be great to come up with a way to make
    # ElemwiseCategoricalStep  more general (handling more complex elementwise
    # variables)
    def __init__(self, vars, model=None):
        model = modelcontext(model)
        #self.sh = ones(vars.dshape, vars.dtype)
        self.vars = vars
        shared = make_shared_replacements(vars, model)
        super(sampleS, self).__init__(vars, shared)

    def astep(self, q, logp):
        return q
        #p = array([logp(v * self.sh) for v in self.values])
        #return categorical(p, self.var.dshape)


class sampleX(ArrayStepShared):
    """
    Use forward sampling (equation 10) to sample a realization of S_t, t=1,...,T_n
    given Q, B, and X constant.
    """
    # TODO: It would be great to come up with a way to make
    # ElemwiseCategoricalStep  more general (handling more complex elementwise
    # variables)
    def __init__(self, vars, model=None):
        model = modelcontext(model)
        #self.sh = ones(vars.dshape, vars.dtype)
        self.vars = vars
        shared = make_shared_replacements(vars, model)
        super(sampleX, self).__init__(vars, shared)

    def astep(self, q, logp):
        return q
        #p = array([logp(v * self.sh) for v in self.values])
        #return categorical(p, self.var.dshape)




## Main

In [12]:
model = pm.Model()

with model:
    Q_raw = DiscreteObsMJP_unif_prior('Q_raw', n=M, shape=(M,M-1))
    
    S = HMM_Blank('S', shape=(Tn,1) )
    C = DiscreteObsMJP('C', Q_raw=Q_raw, S=S, observed_jumps=observed_jumps)

    B = Beta('B', alpha = 1, beta = 1, shape=(K,M))
    X = Comorbidities('X', S=S, B=B, shape=(K, Tn), observed = X_input)

    #Z = Beta('Z')
    #L = Beta('L')
    #O = Observations('O', X=X, Z=Z, L=L, shape=(D, T_n))

with model:
    step1 = Metropolis(vars=[Q_raw])
    step2 = sampleS(vars=[S])
    step3 = Metropolis(vars=[B])


In [13]:
%debug

ERROR: No traceback has been produced, nothing to debug.
